## <font color='blue'>This Jupyter Notebook covers data acquisition from College Confidential</font>

In [1]:
# table goes here

### 1.1 Importing packages 

In [2]:
from bs4 import BeautifulSoup
from datetime import datetime 
from helpers import bot_policy
import json
import os
import requests
import sys

#### 1.1.a saving College Confidential to variable for later use

In [3]:
cc = 'https://www.collegeconfidential.com'

In [4]:
# check current working directory for future debugging
os.getcwd()

'C:\\Users\\Meehir\\Documents\\GitHub\\project-4-public\\notebooks'

### 1.2 Checking website policy using [bot_policy](helpers.py) function

In [5]:
bot_policy(cc)

User-agent: *

Disallow: /cgi-bin

Disallow: /wp-admin

Disallow: /wp-includes

Disallow: /wp-content/plugins

Disallow: /wp-content/cache

Disallow: /wp-content/themes

Disallow: /vibe

Disallow: /trackback

Disallow: /feed

Disallow: /comments

Disallow: /category/*/*

Disallow: */trackback

Disallow: */feed

Disallow: */comments

Disallow: /*?*

Disallow: /*?

Disallow:

Disallow:

Disallow: /

Disallow: /


### 1.3 After navigating to Penn forum, get page response and figure out API call structure by going to the next page

In [6]:
print("Page 1 status is {}".
      format(requests.
             get('https://talk.collegeconfidential.com/university-pennsylvania/').
             status_code))
print("Page 2 status is {}".
      format(requests.
             get('https://talk.collegeconfidential.com/university-pennsylvania//p2').
             status_code))

Page 1 status is 200
Page 2 status is 200


### 1.4 Checking if the same page suffix will work for page 1
> `https://talk.collegeconfidential.com/university-pennsylvania//p1`

#### *If this works, data can be scraped in a loop vs. a more complicated implementation*

In [7]:
penn_url = 'https://talk.collegeconfidential.com/university-pennsylvania//p1'
print("Page status is {}".format(requests.get(penn_url).status_code))

Page status is 200


### 1.5 Now grabbing the page and printing to check if it's working as expected,then making BeautifulSoup object

In [8]:
response = requests.get(penn_url)
response.status_code
page = response.text
print(page)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<style>
    .DataList.CategoryList .Options {
        display: none;
    }
</style><style>

ul.AcceptedAnswers .Item {
   background: transparent;
}

ul.AcceptedAnswers {
border: solid thick blue;
background-color: #eeeeee;
border-radius: 15px;
-webkit-border-radius: 15px;
-moz-border-radius: 15px;
}
</style><style>
    li.QnA-UnansweredQuestions 
    { 
        display:none;
        
    }
</style><title>University of Pennsylvania — College Confidential</title>
<link rel="alternate" type="application/rss+xml" title="University of Pennsylvania — College Confidential" href="https://talk.collegeconfidential.com/university-pennsylvania//feed.rss" />
<link rel="next" href="https://talk.collegeconfidential.com/university-pennsylvania//p2" />
<link rel="stylesheet" href="/applications/dashboard/design/style.css?v=5fad8e9f" media="all" static="1" />
<link rel="stylesheet" href=

### 1.6 Making a list of page urls which will be used later

In [9]:
# by inspection, there are 250 pages
page_list = []
for i in range(1, 251):
    page_list.append('https://talk.collegeconfidential.com/university-pennsylvania//p{}'.format(i))

### 1.7 Importing functions for scraping from [helpers.py](helpers.py) function
    > one_page_topics()
    > page_comments()
    > thread_comments()
    > get_forum_comms()

In [10]:
from helpers import get_forum_comms, make_scrape_list

#### Uncomment top of cell if getting data for the first time. Otherwise, load from json file

In [11]:
#topics_json = get_all_topics(page_list)

#with open("penn_topics.json", 'w', encoding='utf-8') as outfile:
    #json.dump(topics_json, outfile)
    
with open('C:\\Users\\Meehir\\Documents\\GitHub\\project-4-public\\data\\json_files\\penn_topics.json', 'r') as f:
    topics_json = json.load(f)
    f.close()

In [12]:
make_scrape_list(topics_json, 100)

get_forum_comms(topics_json[0:100], 0, 100)
get_forum_comms(topics_json[100:200], 100, 200)
get_forum_comms(topics_json[200:300], 200, 300)
get_forum_comms(topics_json[300:400], 300, 400)
get_forum_comms(topics_json[400:500], 400, 500)
get_forum_comms(topics_json[500:600], 500, 600)
get_forum_comms(topics_json[600:700], 600, 700)
get_forum_comms(topics_json[700:800], 700, 800)
get_forum_comms(topics_json[800:900], 800, 900)
get_forum_comms(topics_json[900:1000], 900, 1000)
get_forum_comms(topics_json[1000:1100], 1000, 1100)
get_forum_comms(topics_json[1100:1200], 1100, 1200)
get_forum_comms(topics_json[1200:1300], 1200, 1300)
get_forum_comms(topics_json[1300:1400], 1300, 1400)
get_forum_comms(topics_json[1400:1500], 1400, 1500)
get_forum_comms(topics_json[1500:1600], 1500, 1600)
get_forum_comms(topics_json[1600:1700], 1600, 1700)
get_forum_comms(topics_json[1700:1800], 1700, 1800)
get_forum_comms(topics_json[1800:1900], 1800, 1900)
get_forum_comms(topics_json[1900:2000], 1900, 2000)
ge

In [13]:
len(topics_json[0:100])

100

In [14]:
#get_forum_comms(topics_json[0:99], 0, 99)

In [15]:
#get_forum_comms(topics_json[100:199], 100, 199)

In [16]:
#get_forum_comms(topics_json[200:299], 200, 299)

In [17]:
#get_forum_comms(topics_json[300:399], 300, 399)

### After a few batches, I realized I was missing 1 item per batch. So, the first few batches have a length of 99 and may be confusingly named

In [18]:
# sanity check
len(topics_json[0:100])

100

#### If acquiring data, the cells below would be uncommented and run. For example, using 
> get_forum_comms(topics_json[2000:2100], 2000, 2100)

In [ ]:
#get_forum_comms(topics_json[2000:2100], 2000, 2100)

In [19]:
#get_forum_comms(topics_json[400:500], 400, 500)

In [20]:
#get_forum_comms(topics_json[500:600], 500, 600)

In [21]:
#get_forum_comms(topics_json[600:700], 600, 700)

In [22]:
#get_forum_comms(topics_json[700:800], 700, 800)

In [23]:
#get_forum_comms(topics_json[800:900], 800, 900)

In [24]:
#get_forum_comms(topics_json[900:1000], 900, 1000)

In [25]:
#get_forum_comms(topics_json[1000:1100], 1000, 1100)

In [26]:
#get_forum_comms(topics_json[1100:1200], 1100, 1200)

In [27]:
#get_forum_comms(topics_json[1200:1300], 1200, 1300)

In [28]:
#get_forum_comms(topics_json[1300:1400], 1300, 1400)

In [29]:
#get_forum_comms(topics_json[1400:1500], 1400, 1500)

In [30]:
#get_forum_comms(topics_json[1500:1600], 1500, 1600)

In [31]:
#get_forum_comms(topics_json[1600:1700], 1600, 1700)

In [32]:
#get_forum_comms(topics_json[1700:1800], 1700, 1800)

In [33]:
#get_forum_comms(topics_json[1800:1900], 1800, 1900)

In [34]:
#get_forum_comms(topics_json[1900:2000], 1900, 2000)

In [35]:
#get_forum_comms(topics_json[2000:2100], 2000, 2100)

In [36]:
#get_forum_comms(topics_json[2100:2200], 2100, 2200)

In [37]:
#get_forum_comms(topics_json[2200:2300], 2200, 2300)

In [38]:
#get_forum_comms(topics_json[2300:2400], 2300, 2400)

In [39]:
#get_forum_comms(topics_json[2400:2500], 2400, 2500)

In [40]:
#get_forum_comms(topics_json[2500:2600], 2500, 2600)

In [41]:
#get_forum_comms(topics_json[2600:2700], 2600, 2700)

In [42]:
#get_forum_comms(topics_json[2700:2800], 2700, 2800)

In [43]:
#get_forum_comms(topics_json[2800:2900], 2800, 2900)

In [44]:
#get_forum_comms(topics_json[2900:3000], 2900, 3000)

In [45]:
#get_forum_comms(topics_json[3000:3100], 3000, 3100)

In [46]:
#get_forum_comms(topics_json[3100:3200], 3100, 3200)

In [47]:
#get_forum_comms(topics_json[3200:3300], 3200, 3300)

In [48]:
#get_forum_comms(topics_json[3300:3400], 3300, 3400)

In [49]:
#get_forum_comms(topics_json[3400:3500], 3400, 3500)

In [50]:
#get_forum_comms(topics_json[3500:3600], 3500, 3600)

In [51]:
#get_forum_comms(topics_json[3600:3700], 3600, 3700)

In [52]:
#get_forum_comms(topics_json[3700:3800], 3700, 3800)

In [53]:
#get_forum_comms(topics_json[3800:3900], 3800, 3900)

In [54]:
#get_forum_comms(topics_json[3900:4000], 3900, 4000)

In [55]:
#get_forum_comms(topics_json[4000:4100], 4000, 4100)

In [56]:
#get_forum_comms(topics_json[4100:4200], 4100, 4200)

In [57]:
#get_forum_comms(topics_json[4200:4300], 4200, 4300)

In [58]:
#get_forum_comms(topics_json[4300:4400], 4300, 4400)

In [59]:
#get_forum_comms(topics_json[4400:4500], 4400, 4500)

In [60]:
#get_forum_comms(topics_json[4500:4600], 4500, 4600)

In [61]:
#get_forum_comms(topics_json[4600:4700], 4600, 4700)

In [62]:
#get_forum_comms(topics_json[4700:4800], 4700, 4800)

In [63]:
#get_forum_comms(topics_json[4800:4900], 4800, 4900)

In [64]:
#get_forum_comms(topics_json[4900:4993], 4900, 4993)